In [ ]:
import os
import sys
from nectarpy import NectarClient

In [ ]:
API_SECRET = "0x57bafe3ca126df877d2a9e38c121759a3dfd0fdbb608533d31816a31252bc2e0"

In [ ]:
nectar_client = NectarClient(API_SECRET,'localhost')

In [ ]:
#### Aggregate type: Linear Regression, Mean, Count, Variance, Minimum, Maximum, Sum

In [ ]:
def pre_compute_func():
    import pandas as pd
    import logging as logger
    df = pd.read_csv("/app/data/worker-data.csv")

    # Lọc dữ liệu: nam giới có tuổi > 50
    filtered = df[(df["gender"] == "MALE") & (df["age"] > 50)]
    logger.error(f'filtered ======> {filtered}')
    # Tính trung bình nhịp tim
    if not filtered.empty:
        heart_rate_mean = filtered["heart_rate"].mean()
    else:
        heart_rate_mean = 0.0

    return {"sum": filtered["heart_rate"].sum(), "count": len(filtered), "mean": heart_rate_mean}

In [ ]:
def main_func(list_of_partial_result):
    total_heart = sum(d.get("heart_rate_sum", 0) for d in list_of_partial_result)
    total_count = sum(d.get("count", 0) for d in list_of_partial_result)
    return {
        "avg_heart_rate": total_heart / total_count if total_count else 0,
        "total_count": total_count
    }

In [ ]:
# def main_func(list_of_partial_result):
#     import pandas as pd

#     # Bước 1: Gộp tất cả các DataFrame lại
#     all_data = pd.concat(list_of_partial_result, ignore_index=True)

#     # Bước 2: Tính toán trên toàn bộ dữ liệu
#     heart_rate_sum = all_data["heart_rate"].sum()
#     total_count = len(all_data)
#     heart_rate_mean = all_data["heart_rate"].mean() if total_count else 0.0

#     return {
#         "avg_heart_rate": float(heart_rate_mean),
#         "total_heart_rate": int(heart_rate_sum),
#         "total_count": int(total_count)
#     }

In [ ]:
#Single data node

In [ ]:
result = nectar_client.byoc_query(
    pre_compute_func = pre_compute_func,
    main_func = main_func, 
    is_separate_data = True,
    bucket_ids=[61967009922181838801239149599260914605694386523894612558900626834098461111511,50457995461806009622957406495585309024464525219123482152376011456519935261339],
    policy_indexes=[0,0],
    use_allowlists=[True,True],
    access_indexes=[0,0]
)

In [ ]:
result

In [ ]:
print(nectar_client.get_result(result))

In [ ]:
### Multiple data node

In [ ]:
print(result)